# RSI Divergence

In [ ]:
!pip install plotly
!pip install "jupyterlab>=3" "ipywidgets>=7.6"
!pip install nbformat 

In [31]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta

In [112]:
data = yf.download('SPY', '2023-01-01' ,'2023-06-28',interval='1h')
data.columns = ['open', 'high', 'low', 'close','adj_close','volume']
data.shape

[*********************100%***********************]  1 of 1 completed


(847, 6)

In [131]:
data['RSI'] = data.ta.rsi(length=20)
data.tail(10)

,index,Datetime,open,high,low,close,adj_close,volume,RSI,pivot,RSIpivot,pointpos,RSIpointpos
837,837,2023-06-26 13:30:00-04:00,432.600006,432.714996,431.750000,432.114990,432.114990,5184256,40.734975,0,0,NaN,NaN
838,838,2023-06-26 14:30:00-04:00,432.109985,432.920013,432.040100,432.639008,432.639008,6166487,42.733206,0,0,NaN,NaN
839,839,2023-06-26 15:30:00-04:00,432.640015,432.850006,431.190002,431.429993,431.429993,16895156,39.498807,1,1,431.189002,25.190064
840,840,2023-06-27 09:30:00-04:00,432.619995,433.299988,431.880005,432.799988,432.799988,11742969,44.508576,0,0,NaN,NaN
841,841,2023-06-27 10:30:00-04:00,432.809998,433.929993,432.540009,433.829987,433.829987,8740714,47.921351,0,0,NaN,NaN
842,842,2023-06-27 11:30:00-04:00,433.869995,434.720001,433.763611,434.640015,434.640015,6126334,50.444336,0,0,NaN,NaN
843,843,2023-06-27 12:30:00-04:00,434.640015,435.690002,434.494995,435.529999,435.529999,6547295,53.073581,0,0,NaN,NaN
844,844,2023-06-27 13:30:00-04:00,435.540009,436.589996,435.540009,436.484985,436.484985,7552342,55.726783,0,0,NaN,NaN
845,845,2023-06-27 14:30:00-04:00,436.489990,436.769012,436.089996,436.739990,436.739990,12601638,56.419369,0,0,NaN,NaN
846,846,2023-06-27 15:30:00-04:00,436.739990,436.809998,436.079987,436.149994,436.149994,11891918,54.348753,0,0,NaN,NaN


In [132]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close']), row=1, col=1)
fig.append_trace(go.Scatter(
    x=data.index,
    y=data['RSI'],
), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [133]:
data.reset_index(inplace=True)

In [134]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

def RSIpivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.RSI[l]>df1.RSI[i]):
            pividlow=0
        if(df1.RSI[l]<df1.RSI[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0 


In [147]:
data['pivot'] = data.apply(lambda x: pivotid(data, x.name,12,12), axis=1)
data['RSIpivot'] = data.apply(lambda x: RSIpivotid(data, x.name, 12, 12), axis=1)

In [148]:
def pointpos(x):
    if x['pivot']==1:
        return x['low']-1e-3
    elif x['pivot']==2:
        return x['high']+1e-3
    else:
        return np.nan

def RSIpointpos(x):
    if x['RSIpivot']==1:
        return x['RSI']-1
    elif x['RSIpivot']==2:
        return x['RSI']+1
    else:
        return np.nan

data['pointpos'] = data.apply(lambda row: pointpos(row), axis=1)
data['RSIpointpos'] = data.apply(lambda row: RSIpointpos(row), axis=1)
#df[df.RSIpivot==1].count()

In [149]:
data[['pointpos','RSIpointpos']].dropna()

,pointpos,RSIpointpos
29,393.701012,75.840526
124,408.161004,69.133684
132,400.278999,46.772190
174,416.490990,62.419144
203,415.050988,59.793284
216,414.001000,58.163238
337,393.451012,51.553349
356,396.471001,57.970220
410,393.689002,46.583143
570,417.620995,67.172480


In [150]:
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close']), row=1, col=1)

fig.add_scatter(x=data.index, y=data['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=1, col=1)

fig.append_trace(go.Scatter(x=data.index, y=data['RSI']), row=2, col=1)
fig.add_scatter(x=data.index, y=data['RSIpointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=2, col=1)


fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [151]:
data

,level_0,index,Datetime,open,high,low,close,adj_close,volume,RSI,pivot,RSIpivot,pointpos,RSIpointpos
0,0,0,2023-01-03 09:30:00-05:00,384.369995,386.429993,380.144989,380.940002,380.940002,18966193,NaN,0,0,NaN,NaN
1,1,1,2023-01-03 10:30:00-05:00,380.890015,382.329987,379.480011,380.029999,380.029999,10055232,NaN,0,0,NaN,NaN
2,2,2,2023-01-03 11:30:00-05:00,380.038208,380.049988,378.429993,378.940002,378.940002,8265613,NaN,0,0,NaN,NaN
3,3,3,2023-01-03 12:30:00-05:00,378.964996,380.019989,377.830994,379.519989,379.519989,6272148,NaN,0,0,NaN,NaN
4,4,4,2023-01-03 13:30:00-05:00,379.519989,380.230011,378.790009,378.989990,378.989990,5515490,NaN,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
842,842,842,2023-06-27 11:30:00-04:00,433.869995,434.720001,433.763611,434.640015,434.640015,6126334,50.444336,0,0,NaN,NaN
843,843,843,2023-06-27 12:30:00-04:00,434.640015,435.690002,434.494995,435.529999,435.529999,6547295,53.073581,0,0,NaN,NaN
844,844,844,2023-06-27 13:30:00-04:00,435.540009,436.589996,435.540009,436.484985,436.484985,7552342,55.726783,0,0,NaN,NaN
845,845,845,2023-06-27 14:30:00-04:00,436.489990,436.769012,436.089996,436.739990,436.739990,12601638,56.419369,0,0,NaN,NaN


In [155]:
import numpy as np
from matplotlib import pyplot
backcandles= 72

#candleid = 8800
candleid = 200

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

maximRSI = np.array([])
minimRSI = np.array([])
xxminRSI = np.array([])
xxmaxRSI = np.array([])

for i in range(candleid-backcandles, candleid+1):
    if data.iloc[i].pivot == 1:
        minim = np.append(minim, data.iloc[i].low)
        xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
    if data.iloc[i].pivot == 2:
        maxim = np.append(maxim, data.iloc[i].high)
        xxmax = np.append(xxmax, i) # df.iloc[i].name
    if data.iloc[i].RSIpivot == 1:
        minimRSI = np.append(minimRSI, data.iloc[i].RSI)
        xxminRSI = np.append(xxminRSI, data.iloc[i].name)
    if data.iloc[i].RSIpivot == 2:
        maximRSI = np.append(maximRSI, data.iloc[i].RSI)
        xxmaxRSI = np.append(xxmaxRSI, data.iloc[i].name)
        
slmin, intercmin = np.polyfit(xxmin, minim,1)
slmax, intercmax = np.polyfit(xxmax, maxim,1)
slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)

print(slmin, slmax, slminRSI, slmaxRSI)


dfpl = data[candleid-backcandles-5:candleid+backcandles]
fig = make_subplots(rows=2, cols=1)
fig.append_trace(go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close']), row=1, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot", row=1, col=1)
fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'), row=1, col=1)
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'), row=1, col=1)

fig.append_trace(go.Scatter(x=dfpl.index, y=dfpl['RSI']), row=2, col=1)
fig.add_scatter(x=dfpl.index, y=dfpl['RSIpointpos'], mode="markers",
                marker=dict(size=2, color="MediumPurple"),
                name="pivot", row=2, col=1)
fig.add_trace(go.Scatter(x=xxminRSI, y=slminRSI*xxminRSI + intercminRSI, mode='lines', name='min slope'), row=2, col=1)
fig.add_trace(go.Scatter(x=xxmaxRSI, y=slmaxRSI*xxmaxRSI + intercmaxRSI, mode='lines', name='max slope'), row=2, col=1)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

0.10111535630276249 -0.07913075322690052 -0.11553301315988562 -0.5382862814032674


# Fitting pivots into slopes

In [171]:
def divsignal(x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    if maxim.size<2 or minim.size<2 or maximRSI.size<2 or minimRSI.size<2:
        return 0
    
    slmin, intercmin = np.polyfit(xxmin, minim,1)
    slmax, intercmax = np.polyfit(xxmax, maxim,1)
    slminRSI, intercminRSI = np.polyfit(xxminRSI, minimRSI,1)
    slmaxRSI, intercmaxRSI = np.polyfit(xxmaxRSI, maximRSI,1)
    
    
    if slmin > 1e-4 and slmax > 1e-4 and slmaxRSI <-0.1:
        return 1
    elif slmin < -1e-4 and slmax < -1e-4 and slminRSI > 0.1:
        return 2
    else:
        return 0

data['divSignal'] = data.apply(lambda row: divsignal(row,30), axis=1)
data

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time,open,high,low,close,volume,RSI,pivot,pointpos,RSIpivot,RSIpointpos,divSignal
0,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996,0.000000,0,NaN,0,NaN,0
1,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992,0.000000,0,NaN,0,NaN,0
2,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992,3.039362,0,NaN,0,NaN,0
3,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083,24.673745,0,NaN,0,NaN,0
4,05.05.2003 17:00:00.000,1.12718,1.13019,1.12657,1.12804,373668.2930,27.632188,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,27.08.2003 13:00:00.000,1.09137,1.09194,1.08862,1.09009,326632.7051,46.592481,0,NaN,0,NaN,0
496,27.08.2003 17:00:00.000,1.09025,1.09042,1.08650,1.08860,375893.1992,44.627474,0,NaN,0,NaN,0
497,27.08.2003 21:00:00.000,1.08856,1.08882,1.08418,1.08589,102160.1992,41.238289,0,NaN,0,NaN,0
498,28.08.2003 01:00:00.000,1.08575,1.08627,1.08415,1.08507,102962.5000,40.247370,0,NaN,0,NaN,0


In [174]:
#dfpl[dfpl.divSignal==1].count()
data[data.divSignal==1]

,time,open,high,low,close,volume,RSI,pivot,pointpos,RSIpivot,RSIpointpos,divSignal
71,20.05.2003 21:00:00.000,1.17059,1.17453,1.16976,1.17225,93439.0000,67.068084,2,1.17553,2,68.068084,1
72,21.05.2003 01:00:00.000,1.17204,1.17287,1.17104,1.17167,94478.8008,66.062121,0,NaN,0,NaN,1
73,21.05.2003 05:00:00.000,1.17140,1.17295,1.16990,1.17148,91076.7988,65.716160,0,NaN,0,NaN,1
74,21.05.2003 09:00:00.000,1.17130,1.17243,1.16754,1.16859,91135.4004,60.548589,0,NaN,0,NaN,1
75,21.05.2003 13:00:00.000,1.16835,1.17405,1.16617,1.17058,306601.0039,62.711825,0,NaN,0,NaN,1
76,21.05.2003 17:00:00.000,1.17070,1.17178,1.16661,1.16746,373083.8047,57.422635,0,NaN,0,NaN,1
77,21.05.2003 21:00:00.000,1.16724,1.16813,1.16244,1.16541,94816.8008,54.204267,0,NaN,0,NaN,1
78,22.05.2003 01:00:00.000,1.16531,1.16592,1.16237,1.16426,94609.8008,52.437806,0,NaN,1,51.437806,1
79,22.05.2003 05:00:00.000,1.16439,1.16548,1.16192,1.16458,88648.5996,52.895472,1,1.16092,0,NaN,1
80,22.05.2003 09:00:00.000,1.16517,1.17179,1.16496,1.17167,90969.0000,61.654580,0,NaN,0,NaN,1


# pivot points levels instead of slopes

In [14]:
dfpl = df[0:500]
def divsignal2(x, nbackcandles):
    backcandles=nbackcandles 
    candleid = int(x.name)

    closp = np.array([])
    xxclos = np.array([])
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    maximRSI = np.array([])
    minimRSI = np.array([])
    xxminRSI = np.array([])
    xxmaxRSI = np.array([])

    for i in range(candleid-backcandles, candleid+1):
        closp = np.append(closp, df.iloc[i].close)
        xxclos = np.append(xxclos, i)
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
        if df.iloc[i].RSIpivot == 1:
            minimRSI = np.append(minimRSI, df.iloc[i].RSI)
            xxminRSI = np.append(xxminRSI, df.iloc[i].name)
        if df.iloc[i].RSIpivot == 2:
            maximRSI = np.append(maximRSI, df.iloc[i].RSI)
            xxmaxRSI = np.append(xxmaxRSI, df.iloc[i].name)

    slclos, interclos = np.polyfit(xxclos, closp,1)
    
    if slclos > 1e-4 and (maximRSI.size<2 or maxim.size<2):
        return 0
    if slclos < -1e-4 and (minimRSI.size<2 or minim.size<2):
        return 0
# signal decisions here !!!
    if slclos > 1e-4:
        if maximRSI[-1]<maximRSI[-2] and maxim[-1]>maxim[-2]:
            return 1
    elif slclos < -1e-4:
        if minimRSI[-1]>minimRSI[-2] and minim[-1]<minim[-2]:
            return 2
    else:
        return 0

dfpl['divSignal2'] = dfpl.apply(lambda row: divsignal2(row,30), axis=1)
dfpl

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time,open,high,low,close,volume,RSI,pivot,RSIpivot,pointpos,RSIpointpos,divSignal2
0,04.05.2003 21:00:00.000,1.12354,1.12354,1.12166,1.12274,95533.0976,NaN,0,0,NaN,NaN,0.0
1,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996,NaN,0,0,NaN,NaN,0.0
2,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992,NaN,0,0,NaN,NaN,0.0
3,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992,NaN,0,0,NaN,NaN,0.0
4,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083,NaN,0,0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,27.08.2003 09:00:00.000,1.08883,1.09298,1.08833,1.09123,106726.7012,49.073492,2,2,1.09398,50.073492,0.0
496,27.08.2003 13:00:00.000,1.09137,1.09194,1.08862,1.09009,326632.7051,47.437055,0,0,NaN,NaN,0.0
497,27.08.2003 17:00:00.000,1.09025,1.09042,1.08650,1.08860,375893.1992,45.310309,0,0,NaN,NaN,0.0
498,27.08.2003 21:00:00.000,1.08856,1.08882,1.08418,1.08589,102160.1992,41.652611,0,0,NaN,NaN,0.0


In [16]:
#dfpl[dfpl.divSignal2==2].count()
dfpl[dfpl.divSignal2==1]

,time,open,high,low,close,volume,RSI,pivot,RSIpivot,pointpos,RSIpointpos,divSignal2
72,20.05.2003 21:00:00.000,1.17059,1.17453,1.16976,1.17225,93439.0000,67.727389,2,2,1.17553,68.727389,1.0
73,21.05.2003 01:00:00.000,1.17204,1.17287,1.17104,1.17167,94478.8008,66.635960,0,0,NaN,NaN,1.0
74,21.05.2003 05:00:00.000,1.17140,1.17295,1.16990,1.17148,91076.7988,66.259266,0,0,NaN,NaN,1.0
75,21.05.2003 09:00:00.000,1.17130,1.17243,1.16754,1.16859,91135.4004,60.643690,0,0,NaN,NaN,1.0
76,21.05.2003 13:00:00.000,1.16835,1.17405,1.16617,1.17058,306601.0039,62.970875,0,0,NaN,NaN,1.0
77,21.05.2003 17:00:00.000,1.17070,1.17178,1.16661,1.16746,373083.8047,57.254599,0,0,NaN,NaN,1.0
78,21.05.2003 21:00:00.000,1.16724,1.16813,1.16244,1.16541,94816.8008,53.798937,0,0,NaN,NaN,1.0
79,22.05.2003 01:00:00.000,1.16531,1.16592,1.16237,1.16426,94609.8008,51.906291,0,1,NaN,50.906291,1.0
80,22.05.2003 05:00:00.000,1.16439,1.16548,1.16192,1.16458,88648.5996,52.408017,1,0,1.16092,NaN,1.0
81,22.05.2003 09:00:00.000,1.16517,1.17179,1.16496,1.17167,90969.0000,61.893476,0,0,NaN,NaN,1.0
